In [1]:
import json
from collections import defaultdict

from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, clear_output
import ipywidgets as widgets
import plotly.graph_objs as go
import plotly
import pandas as pd

init_notebook_mode(connected=True)

In [2]:
ns_data = None
with open('ntwk_stats/ntwk_stats.csv', 'r') as ntsd:
    ns_data = pd.read_csv(ntsd)

In [3]:
class NtwkStatsVis:
    def __init__(self, ntsd_df):
        
        self.data = ntsd_df

        r_vals = sorted(self.data['r'].unique())
        p_vals = sorted(self.data['p'].unique())
        
        self.data['cheeger_l'] = self.data['eig2'] / 2
        self.data['cheeger_u'] = (2 * self.data['eig2']) ** 0.5
        
        # Welfare by p
        self.p_val = widgets.Select(
                                    options=p_vals,
                                    description='Prob info trans',
                                    disabled=False)
        self.r_val = widgets.Select(
                                    options=r_vals,
                                    description='Resources per vertex',
                                    disabled=False)
        
        self.ntwk_types = widgets.SelectMultiple(
                                    options=['rl', 'ws', 'cm', 'kg'],
                                    description='Network type',
                                    disabled=False)
        
        self.param_selection = widgets.Select(
                                    options=['diam', 'apl', 'clust_coeff', 'loc_eff',
                                             'eig2', 'cheeger_l', 'cheeger_u'],
                                    description='Param to plot',
                                    disabled=False)

        # Update button
        self.plot_button = widgets.Button(
                                    description='Plot',
                                    disabled=False,
                                    button_style='',
                                    tooltip='Plots by selected parameters')
        self.plot_button.on_click(self.plot)
        
        self.visuals = [self.p_val, self.r_val,
                        self.ntwk_types, self.param_selection,
                        self.plot_button]
        
        self.render_visuals()
        
    def render_visuals(self):

        # Render widgets
        for vis in self.visuals:
            display(vis)

    def plot(self, button):
        
        # Reset widgets
        clear_output()
        self.render_visuals()
        
        p_val = self.p_val.value
        r_val = self.r_val.value
        
        param = self.param_selection.value
        ntwk_types = self.ntwk_types.value
        
        self.plot_simple_param(p_val, r_val, ntwk_types, param)
    
    def select_data(self, p_val, r_val, ntwk):
        p_val_cond = self.data['p'] == p_val
        r_val_cond = self.data['r'] == r_val
        ntwk_cond = self.data['ntwk'] == ntwk
        return self.data[p_val_cond & r_val_cond & ntwk_cond]
    
    def plot_simple_param(self, p_val, r_val, ntwk_types, param):
        fig = go.Figure()

        colors = plotly.colors.DEFAULT_PLOTLY_COLORS
        
        for idx, ntwk in enumerate(ntwk_types):
            color = colors[idx % len(colors)]
            
            indiv_fig = go.Figure()
            
            plot_data = self.select_data(p_val, r_val, ntwk)
            
            x_vals = plot_data[param]
            y_vals = plot_data['final_util']

            fig.add_trace(go.Scatter(x=x_vals, y=y_vals,
                                     mode='markers', hoverinfo='skip',
                                     marker=dict(color=color),
                                     name=f'ntwk={ntwk}'))
                                     
            indiv_fig.add_trace(go.Scatter(x=x_vals, y=y_vals,
                         mode='markers', hoverinfo='skip',
                         marker=dict(color=color),
                         name=f'ntwk={ntwk}'))
            indiv_fig.layout.update(title=f'{param} vs final util (p={p_val}, r={r_val}, ntwk={ntwk})',
                        xaxis=dict(title=param), yaxis=dict(title='Final utility (sum)'),
                        plot_bgcolor='rgba(0,0,0,0)')
            iplot(indiv_fig)

        plot_title = f'{param} vs final util (p={p_val}, r={r_val} ntwk comparison)'
        fig.layout.update(title=plot_title, showlegend=True,
            xaxis=dict(title=param), yaxis=dict(title='final utility (sum)'),
            plot_bgcolor='rgba(0,0,0,0)')

        iplot(fig)

In [4]:
vis = NtwkStatsVis(ns_data)

Select(description='Prob info trans', index=1, options=(0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0), val…

Select(description='Resources per vertex', index=2, options=(2, 4, 8, 16, 32, 64, 128), value=8)

SelectMultiple(description='Network type', index=(0, 1, 2, 3), options=('rl', 'ws', 'cm', 'kg'), value=('rl', …

Select(description='Param to plot', index=5, options=('diam', 'apl', 'clust_coeff', 'loc_eff', 'eig2', 'cheege…

Button(description='Plot', style=ButtonStyle(), tooltip='Plots by selected parameters')